# 1. Design a NIN network

In [2]:
import numpy as np

# 思路
1. 确定输入的维度。
2. 确定卷积的形状。
3. 确定1*1的卷积。
4. 池化。
5. 输出。

In [8]:
# 使用tensorflow的实现。
import tensorflow as tf
from tensorflow.keras import layers
input = np.arange(512*512*32, dtype=float).reshape(512, 512, 32)
output = np.arange(128*128*64, dtype=float).reshape(128, 128, 64)

# 输出的通道数
input_filters = 1028

outputFilters =64
# input_dim=115, use_bias=True,
model = tf.keras.models.Sequential()
model.add(layers.InputLayer(input_shape=(512, 512, 32), name="input", dtype="float32"))

model.add(layers.Conv2D(1028, kernel_size=5,
                        strides=1, activation="relu", padding="same", name="conv2D"))
# 最后的全连接层并不是通过dense来实现的，而是通过1*1的卷积来实现的。这里有点没有想明白。
# 难道不是通过一次1*1的卷积之后，后面的才是2层全连接层吗？
# model.add(layers.Dense(2048, activation="relu"))
# model.add(layers.Dense(2048, activation="relu"))
model.add(layers.Conv2D(1028, kernel_size=1,
                        strides=1, activation="relu", padding="same", name="Conv1m1_0"))
model.add(layers.Conv2D(1028, kernel_size=1,
                        strides=1, activation="relu", padding="same", name="Conv1m1_1"))
# 对每个
model.add(layers.MaxPool2D(pool_size=3, strides=2, name="pooling"))

# input_shape 这里是指定输入的结构，其中第一个是batch，第二个和第三个是图片的长宽，
# 第三个是通道数，如果是RGB那么就是3。
# model.build(input_shape=(1, 512, 512, 32))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2D (Conv2D)              (None, 512, 512, 1028)    823428    
_________________________________________________________________
Conv1m1_0 (Conv2D)           (None, 512, 512, 1028)    1057812   
_________________________________________________________________
Conv1m1_1 (Conv2D)           (None, 512, 512, 1028)    1057812   
_________________________________________________________________
pooling (MaxPooling2D)       (None, 255, 255, 1028)    0         
Total params: 2,939,052
Trainable params: 2,939,052
Non-trainable params: 0
_________________________________________________________________


In [35]:
# 参考AlexNet实现。
from tensorflow.keras import layers, models, Model, Sequential
im_height=224
im_width=224
class_num=1000
input_image = layers.Input(shape=(im_height, im_width, 3), dtype="float32")  # output(None, 224, 224, 3)
x = layers.ZeroPadding2D(((1, 2), (1, 2)))(input_image)   #valid和same都不能满足输出，因此需要手动padding处理 output(None, 227, 227, 3)
x = layers.Conv2D(48, kernel_size=11, strides=4, activation="relu")(x)       # output(None, 55, 55, 48)
x = layers.MaxPool2D(pool_size=3, strides=2)(x) #padding默认等于valid  # output(None, 27, 27, 48)
x = layers.Conv2D(128, kernel_size=5, padding="same", activation="relu")(x)  # output(None, 27, 27, 128)
x = layers.MaxPool2D(pool_size=3, strides=2)(x)                              # output(None, 13, 13, 128)
x = layers.Conv2D(192, kernel_size=3, padding="same", activation="relu")(x)  # output(None, 13, 13, 192)
x = layers.Conv2D(192, kernel_size=3, padding="same", activation="relu")(x)  # output(None, 13, 13, 192)
x = layers.Conv2D(128, kernel_size=3, padding="same", activation="relu")(x)  # output(None, 13, 13, 128)
x = layers.MaxPool2D(pool_size=3, strides=2)(x)                              # output(None, 6, 6, 128)

x = layers.Flatten()(x)                         # output(None, 6*6*128=4608)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2048, activation="relu")(x)    # output(None, 2048)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2048, activation="relu")(x)    # output(None, 2048)
x = layers.Dense(class_num)(x)                  # output(None, 5)

predict = layers.Softmax()(x)#将输出转化成为一个概率分布

model = models.Model(inputs=input_image, outputs=predict)
model.summary()
# predict = layers.Softmax()(x)#将输出转化成为一个概率分布

# model = models.Model(inputs=input_image, outputs=predict)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 55, 55, 48)        17472     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 48)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 27, 27, 128)       153728    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 13, 13, 192)       221376

In [4]:
# 梯度使用
import tensorflow as tf

x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
dy_dx = g.gradient(y, x) 
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)
